In [1]:
!pip install pathlib

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install pygments==2.13.0

Defaulting to user installation because normal site-packages is not writeable


In [3]:
!pip install numpy==1.23

Defaulting to user installation because normal site-packages is not writeable


In [4]:
!pip install pytorch_tabular

Defaulting to user installation because normal site-packages is not writeable


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from keras.optimizers import Adam
from keras_nlp.layers import PositionEmbedding

import numpy.random as npr
from sklearn.metrics import mean_squared_error

from generate_data import *
from baseline_results import *
from attention_model_adam import *

from pytorch_tabular import TabularModel
from pytorch_tabular.models import (
    CategoryEmbeddingModelConfig, 
    FTTransformerConfig, 
    TabNetModelConfig, 
    GatedAdditiveTreeEnsembleConfig, 
    TabTransformerConfig, 
    AutoIntConfig
)
from pytorch_tabular.config import DataConfig, OptimizerConfig, TrainerConfig, ExperimentConfig
from pytorch_tabular.models.common.heads import LinearHeadConfig

2023-05-24 03:05:11.063986: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-24 03:05:11.190826: W tensorflow/tsl/platform/default/dso_loader.cc:66] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/slurm/lib64:
2023-05-24 03:05:11.190860: I tensorflow/tsl/cuda/cudart_stub.cc:28] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-05-24 03:05:11.234078: E tensorflow/tsl/lib/monitoring/collection_registry.cc:81] Cannot register 2 metrics with the same name: /tensorflow/core/bfc_allocator_delay
2023-05-24 03:05:12.224834: W tensorflow/tsl/platform/default/dso_loader.cc:66] 

### Data processing

In [6]:
data = pd.read_csv('auto-mpg.csv')
data = data[data['horsepower'] != '?']
data = data[(data['cylinders'] == 4) | (data['cylinders'] == 6) | (data['cylinders'] == 8)  ]
data['horsepower'] = data['horsepower'].astype(int)

In [7]:
data2 = pd.DataFrame()

for i in data.columns:
    if i in ['origin', 'car name']:
        data2[i] = data[i]
    elif i == 'cylinders':
        data2[i] = np.array(data[i]/2 - 2).astype(int)
    elif i == 'mpg':
        data2['y'] = pd.qcut(data[i], 3, labels = False)
    else:
        data2[i] = pd.qcut(data[i], 3, labels = False)

In [45]:
train = data2[data2['origin'] == 1].reset_index(drop=True)
test = data2[data2['origin'] != 1].reset_index(drop=True)

del train['origin']
del train['car name']
del test['origin']
del test['car name']

In [46]:
test['cylinders'] = pd.Categorical(test['cylinders'], categories=[0, 1, 2])
test['displacement'] = pd.Categorical(test['displacement'], categories=[0, 1, 2])

In [47]:
train.columns = ['y'] + ['x'+str(i) for i in range(6)]
test.columns = ['y'] + ['x'+str(i) for i in range(6)]

In [48]:
train = train[['x'+str(i) for i in range(6)] + ['y']]
test = test[['x'+str(i) for i in range(6)] + ['y']]

In [78]:
test['x0'] = test['x0'].astype(int)
test['x1'] = test['x1'].astype(int)

### Baseline

In [12]:
get_baseline_results(train = train, test = test, n_cat = 3, seed = 42)

{'mse_train': [0.13877551020408163,
  0.1469387755102041,
  0.1346938775510204,
  0.10204081632653061],
 'acc_train': [0.8612244897959184,
  0.8530612244897959,
  0.8653061224489796,
  0.8979591836734694],
 'mse_test': [0.34285714285714286,
  0.2785714285714286,
  0.34285714285714286,
  0.3],
 'acc_test': [0.6571428571428571, 0.7214285714285714, 0.6571428571428571, 0.7]}

### Attention

In [13]:
train_attention_model(train = train, test = test, n_cat = 3, embed_dim = 20, ff_dim = 5, num_head = 5, seed = 42)

2023-05-24 03:06:04.368031: W tensorflow/tsl/platform/default/dso_loader.cc:66] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/slurm/lib64:
2023-05-24 03:06:04.368077: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-05-24 03:06:04.368097: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (gl3027.arc-ts.umich.edu): /proc/driver/nvidia/version does not exist
2023-05-24 03:06:04.427825: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/200
14/14 - 4s - loss: 0.1777 - 4s/epoch - 262ms/step
Epoch 2/200
14/14 - 0s - loss: 0.1574 - 57ms/epoch - 4ms/step
Epoch 3/200
14/14 - 0s - loss: 0.1472 - 56ms/epoch - 4ms/step
Epoch 4/200
14/14 - 0s - loss: 0.1412 - 54ms/epoch - 4ms/step
Epoch 5/200
14/14 - 0s - loss: 0.1349 - 55ms/epoch - 4ms/step
Epoch 6/200
14/14 - 0s - loss: 0.1303 - 55ms/epoch - 4ms/step
Epoch 7/200
14/14 - 0s - loss: 0.1283 - 55ms/epoch - 4ms/step
Epoch 8/200
14/14 - 0s - loss: 0.1270 - 56ms/epoch - 4ms/step
Epoch 9/200
14/14 - 0s - loss: 0.1264 - 54ms/epoch - 4ms/step
Epoch 10/200
14/14 - 0s - loss: 0.1258 - 56ms/epoch - 4ms/step
Epoch 11/200
14/14 - 0s - loss: 0.1243 - 55ms/epoch - 4ms/step
Epoch 12/200
14/14 - 0s - loss: 0.1236 - 54ms/epoch - 4ms/step
Epoch 13/200
14/14 - 0s - loss: 0.1234 - 55ms/epoch - 4ms/step
Epoch 14/200
14/14 - 0s - loss: 0.1210 - 55ms/epoch - 4ms/step
Epoch 15/200
14/14 - 0s - loss: 0.1191 - 53ms/epoch - 4ms/step
Epoch 16/200
14/14 - 0s - loss: 0.1162 - 52ms/epoch - 4ms/step
E

Epoch 131/200
14/14 - 0s - loss: 0.0609 - 55ms/epoch - 4ms/step
Epoch 132/200
14/14 - 0s - loss: 0.0612 - 55ms/epoch - 4ms/step
Epoch 133/200
14/14 - 0s - loss: 0.0614 - 54ms/epoch - 4ms/step
Epoch 134/200
14/14 - 0s - loss: 0.0601 - 61ms/epoch - 4ms/step
Epoch 135/200
14/14 - 0s - loss: 0.0603 - 58ms/epoch - 4ms/step
Epoch 136/200
14/14 - 0s - loss: 0.0613 - 56ms/epoch - 4ms/step
Epoch 137/200
14/14 - 0s - loss: 0.0610 - 55ms/epoch - 4ms/step
Epoch 138/200
14/14 - 0s - loss: 0.0612 - 56ms/epoch - 4ms/step
Epoch 139/200
14/14 - 0s - loss: 0.0617 - 55ms/epoch - 4ms/step
Epoch 140/200
14/14 - 0s - loss: 0.0611 - 57ms/epoch - 4ms/step
Epoch 141/200
14/14 - 0s - loss: 0.0601 - 58ms/epoch - 4ms/step
Epoch 142/200
14/14 - 0s - loss: 0.0604 - 56ms/epoch - 4ms/step
Epoch 143/200
14/14 - 0s - loss: 0.0608 - 54ms/epoch - 4ms/step
Epoch 144/200
14/14 - 0s - loss: 0.0605 - 54ms/epoch - 4ms/step
Epoch 145/200
14/14 - 0s - loss: 0.0597 - 54ms/epoch - 4ms/step
Epoch 146/200
14/14 - 0s - loss: 0.0609 

(0.14285714285714285,
 0.8571428571428571,
 0.20714285714285716,
 0.7928571428571428)

### Other baselines

In [16]:
data_config = DataConfig(
    target=['y'], #target should always be a list. Multi-targets are only supported for regression. Multi-Task Classification is not implemented
    continuous_cols=[],
    categorical_cols=[x for x in train.columns if x != 'y'],
)

trainer_config = TrainerConfig(
    batch_size=128,
    max_epochs=200,
    accelerator="auto", # can be 'cpu','gpu', 'tpu', or 'ipu' 
)

optimizer_config = OptimizerConfig()

head_config = LinearHeadConfig(
    layers="", # No additional layer in head, just a mapping layer to output_dim
    dropout=0.1,
    initialization="kaiming"
).__dict__ # Convert to dict to pass to the model config (OmegaConf doesn't accept objects)

In [17]:
### Category embedding

model_config = CategoryEmbeddingModelConfig(
    task="classification",
    layers="64-32",  # Number of nodes in each layer
    activation="ReLU", # Activation between each layers
    learning_rate = 1e-3,
    head = "LinearHead", #Linear Head
    head_config = head_config, # Linear Head Config
)

tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)

tabular_model.fit(train=train)

2023-05-24 03:07:13,706 - {pytorch_tabular.tabular_model:102} - INFO - Experiment Tracking is turned off
[rank: 0] Global seed set to 42
2023-05-24 03:07:13,750 - {pytorch_tabular.tabular_model:465} - INFO - Preparing the DataLoaders
2023-05-24 03:07:13,751 - {pytorch_tabular.tabular_datamodule:286} - INFO - Setting up the datamodule for classification task
2023-05-24 03:07:13,773 - {pytorch_tabular.tabular_model:508} - INFO - Preparing the Model: CategoryEmbeddingModel
/home/kwib/.local/lib/python3.9/site-packages/pytorch_tabular/models/base_model.py:126: UserWarning: Wandb is not installed. Please install wandb to log logits. You can install wandb using pip install wandb or install PyTorch Tabular using pip install pytorch-tabular[all]
  warnings.warn(
/home/kwib/.local/lib/python3.9/site-packages/pytorch_tabular/models/base_model.py:132: UserWarning: Plotly is not installed. Please install plotly to log logits. You can install plotly using pip install plotly or install PyTorch Tabul

┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type                      ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ _backbone        │ CategoryEmbeddingBackbone │  2.9 K │
│ 1 │ _embedding_layer │ Embedding1dLayer          │     48 │
│ 2 │ head             │ LinearHead                │     99 │
│ 3 │ loss             │ CrossEntropyLoss          │      0 │
└───┴──────────────────┴───────────────────────────┴────────┘

Trainable params: 3.1 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 3.1 K                                                                                                
Total estimated model params size (MB): 0

Output()

/home/kwib/.local/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: 
PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. 
Consider increasing the value of the `num_workers` argument` (try 36 which is the number of cpus on this machine) 
in the `DataLoader` init to improve performance.
  rank_zero_warn(

/home/kwib/.local/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: 
PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. 
Consider increasing the value of the `num_workers` argument` (try 36 which is the number of cpus on this machine) 
in the `DataLoader` init to improve performance.
  rank_zero_warn(

2023-05-24 03:07:19,559 - {pytorch_tabular.tabular_model:568} - INFO - Training the model completed
2023-05-24 03:07:19,560 - {pytorch_tabular.tabular_model:1207} - INFO - Loading the best model


In [81]:
pred_df = tabular_model.predict(test)

### Test accuracy
print((pred_df['y'] == pred_df['prediction']).mean())

### Test MSE
print(((pred_df['y'] - pred_df['prediction'])**2).mean())

Output()

0.7642857142857142
0.2357142857142857


In [82]:
### FTTransformer

model_config = FTTransformerConfig(
    task="classification",
    learning_rate = 1e-3,
    head = "LinearHead", #Linear Head
    head_config = head_config, # Linear Head Config
)

tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)

tabular_model.fit(train=train)

/home/kwib/.local/lib/python3.9/site-packages/pytorch_tabular/models/ft_transformer/config.py:229: UserWarning: Ignoring the deprecated arguments, `out_ff_layers`, `out_ff_activation`, `out_ff_dropoout`, and `out_ff_initialization` as head_config is passed.
  warnings.warn(
2023-05-24 03:19:23,384 - {pytorch_tabular.tabular_model:102} - INFO - Experiment Tracking is turned off
[rank: 0] Global seed set to 42
2023-05-24 03:19:23,408 - {pytorch_tabular.tabular_model:465} - INFO - Preparing the DataLoaders
2023-05-24 03:19:23,409 - {pytorch_tabular.tabular_datamodule:286} - INFO - Setting up the datamodule for classification task
2023-05-24 03:19:23,426 - {pytorch_tabular.tabular_model:508} - INFO - Preparing the Model: FTTransformerModel
/home/kwib/.local/lib/python3.9/site-packages/pytorch_tabular/models/base_model.py:126: UserWarning: Wandb is not installed. Please install wandb to log logits. You can install wandb using pip install wandb or install PyTorch Tabular using pip install py

┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type                  ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ _backbone        │ FTTransformerBackbone │  271 K │
│ 1 │ _embedding_layer │ Embedding2dLayer      │  1.0 K │
│ 2 │ _head            │ LinearHead            │     99 │
│ 3 │ loss             │ CrossEntropyLoss      │      0 │
└───┴──────────────────┴───────────────────────┴────────┘

Trainable params: 272 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 272 K                                                                                                
Total estimated model params size (MB): 1

Output()

/home/kwib/.local/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: 
PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. 
Consider increasing the value of the `num_workers` argument` (try 36 which is the number of cpus on this machine) 
in the `DataLoader` init to improve performance.
  rank_zero_warn(

/home/kwib/.local/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: 
PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. 
Consider increasing the value of the `num_workers` argument` (try 36 which is the number of cpus on this machine) 
in the `DataLoader` init to improve performance.
  rank_zero_warn(

2023-05-24 03:19:27,934 - {pytorch_tabular.tabular_model:568} - INFO - Training the model completed
2023-05-24 03:19:27,934 - {pytorch_tabular.tabular_model:1207} - INFO - Loading the best model


In [83]:
pred_df = tabular_model.predict(test)

### Test accuracy
print((pred_df['y'] == pred_df['prediction']).mean())

### Test MSE
print(((pred_df['y'] - pred_df['prediction'])**2).mean())

Output()

0.7071428571428572
0.29285714285714287


In [84]:
### TabTransformer

model_config = TabTransformerConfig(
    task="classification",
    learning_rate = 1e-3,
    head = "LinearHead", #Linear Head
    head_config = head_config, # Linear Head Config
)

tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)

tabular_model.fit(train=train)

/home/kwib/.local/lib/python3.9/site-packages/pytorch_tabular/models/tab_transformer/config.py:219: UserWarning: Ignoring the deprecated arguments, `out_ff_layers`, `out_ff_activation`, `out_ff_dropoout`, and `out_ff_initialization` as head_config is passed.
  warnings.warn(
2023-05-24 03:20:02,755 - {pytorch_tabular.tabular_model:102} - INFO - Experiment Tracking is turned off
[rank: 0] Global seed set to 42
2023-05-24 03:20:02,774 - {pytorch_tabular.tabular_model:465} - INFO - Preparing the DataLoaders
2023-05-24 03:20:02,774 - {pytorch_tabular.tabular_datamodule:286} - INFO - Setting up the datamodule for classification task
2023-05-24 03:20:02,791 - {pytorch_tabular.tabular_model:508} - INFO - Preparing the Model: TabTransformerModel
/home/kwib/.local/lib/python3.9/site-packages/pytorch_tabular/models/base_model.py:126: UserWarning: Wandb is not installed. Please install wandb to log logits. You can install wandb using pip install wandb or install PyTorch Tabular using pip install 

┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type                   ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ _backbone        │ TabTransformerBackbone │  271 K │
│ 1 │ _embedding_layer │ Embedding2dLayer       │    816 │
│ 2 │ _head            │ LinearHead             │    579 │
│ 3 │ loss             │ CrossEntropyLoss       │      0 │
└───┴──────────────────┴────────────────────────┴────────┘

Trainable params: 272 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 272 K                                                                                                
Total estimated model params size (MB): 1

Output()

/home/kwib/.local/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: 
PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. 
Consider increasing the value of the `num_workers` argument` (try 36 which is the number of cpus on this machine) 
in the `DataLoader` init to improve performance.
  rank_zero_warn(

/home/kwib/.local/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: 
PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. 
Consider increasing the value of the `num_workers` argument` (try 36 which is the number of cpus on this machine) 
in the `DataLoader` init to improve performance.
  rank_zero_warn(

2023-05-24 03:20:05,752 - {pytorch_tabular.tabular_model:568} - INFO - Training the model completed
2023-05-24 03:20:05,753 - {pytorch_tabular.tabular_model:1207} - INFO - Loading the best model


In [85]:
pred_df = tabular_model.predict(test)

### Test accuracy
print((pred_df['y'] == pred_df['prediction']).mean())

### Test MSE
print(((pred_df['y'] - pred_df['prediction'])**2).mean())

Output()

0.7071428571428572
0.29285714285714287


In [87]:
### AutoInt

model_config = AutoIntConfig(
    task="classification",
    learning_rate = 1e-3,
    head = "LinearHead", #Linear Head
    head_config = head_config, # Linear Head Config
)

tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)

tabular_model.fit(train=train)

2023-05-24 03:20:38,395 - {pytorch_tabular.tabular_model:102} - INFO - Experiment Tracking is turned off
[rank: 0] Global seed set to 42
2023-05-24 03:20:38,418 - {pytorch_tabular.tabular_model:465} - INFO - Preparing the DataLoaders
2023-05-24 03:20:38,419 - {pytorch_tabular.tabular_datamodule:286} - INFO - Setting up the datamodule for classification task
2023-05-24 03:20:38,436 - {pytorch_tabular.tabular_model:508} - INFO - Preparing the Model: AutoIntModel
/home/kwib/.local/lib/python3.9/site-packages/pytorch_tabular/models/base_model.py:126: UserWarning: Wandb is not installed. Please install wandb to log logits. You can install wandb using pip install wandb or install PyTorch Tabular using pip install pytorch-tabular[all]
  warnings.warn(
/home/kwib/.local/lib/python3.9/site-packages/pytorch_tabular/models/base_model.py:132: UserWarning: Plotly is not installed. Please install plotly to log logits. You can install plotly using pip install plotly or install PyTorch Tabular using p

┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type             ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ _backbone        │ AutoIntBackbone  │ 13.8 K │
│ 1 │ _embedding_layer │ Embedding2dLayer │    504 │
│ 2 │ _head            │ LinearHead       │    579 │
│ 3 │ loss             │ CrossEntropyLoss │      0 │
└───┴──────────────────┴──────────────────┴────────┘

Trainable params: 14.8 K                                                                                           
Non-trainable params: 0                                                                                            
Total params: 14.8 K                                                                                               
Total estimated model params size (MB): 0

Output()

/home/kwib/.local/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: 
PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. 
Consider increasing the value of the `num_workers` argument` (try 36 which is the number of cpus on this machine) 
in the `DataLoader` init to improve performance.
  rank_zero_warn(

/home/kwib/.local/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: 
PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. 
Consider increasing the value of the `num_workers` argument` (try 36 which is the number of cpus on this machine) 
in the `DataLoader` init to improve performance.
  rank_zero_warn(

2023-05-24 03:20:41,479 - {pytorch_tabular.tabular_model:568} - INFO - Training the model completed
2023-05-24 03:20:41,479 - {pytorch_tabular.tabular_model:1207} - INFO - Loading the best model


In [88]:
pred_df = tabular_model.predict(test)

### Test accuracy
print((pred_df['y'] == pred_df['prediction']).mean())

### Test MSE
print(((pred_df['y'] - pred_df['prediction'])**2).mean())

Output()

0.36428571428571427
0.6357142857142857


In [89]:
### TabNet

model_config = TabNetModelConfig(
    task="classification",
    learning_rate = 1e-3,
    head = "LinearHead", #Linear Head
    head_config = head_config, # Linear Head Config
)

tabular_model = TabularModel(
    data_config=data_config,
    model_config=model_config,
    optimizer_config=optimizer_config,
    trainer_config=trainer_config,
)

tabular_model.fit(train=train)

2023-05-24 03:21:29,291 - {pytorch_tabular.tabular_model:102} - INFO - Experiment Tracking is turned off
[rank: 0] Global seed set to 42
2023-05-24 03:21:29,311 - {pytorch_tabular.tabular_model:465} - INFO - Preparing the DataLoaders
2023-05-24 03:21:29,311 - {pytorch_tabular.tabular_datamodule:286} - INFO - Setting up the datamodule for classification task
2023-05-24 03:21:29,329 - {pytorch_tabular.tabular_model:508} - INFO - Preparing the Model: TabNetModel
/home/kwib/.local/lib/python3.9/site-packages/pytorch_tabular/models/base_model.py:126: UserWarning: Wandb is not installed. Please install wandb to log logits. You can install wandb using pip install wandb or install PyTorch Tabular using pip install pytorch-tabular[all]
  warnings.warn(
/home/kwib/.local/lib/python3.9/site-packages/pytorch_tabular/models/base_model.py:132: UserWarning: Plotly is not installed. Please install plotly to log logits. You can install plotly using pip install plotly or install PyTorch Tabular using pi

┏━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃   ┃ Name             ┃ Type             ┃ Params ┃
┡━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ 0 │ _embedding_layer │ Identity         │      0 │
│ 1 │ _backbone        │ TabNetBackbone   │  6.5 K │
│ 2 │ _head            │ Identity         │      0 │
│ 3 │ loss             │ CrossEntropyLoss │      0 │
└───┴──────────────────┴──────────────────┴────────┘

Trainable params: 6.5 K                                                                                            
Non-trainable params: 0                                                                                            
Total params: 6.5 K                                                                                                
Total estimated model params size (MB): 0

Output()

/home/kwib/.local/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: 
PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. 
Consider increasing the value of the `num_workers` argument` (try 36 which is the number of cpus on this machine) 
in the `DataLoader` init to improve performance.
  rank_zero_warn(

/home/kwib/.local/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: 
PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. 
Consider increasing the value of the `num_workers` argument` (try 36 which is the number of cpus on this machine) 
in the `DataLoader` init to improve performance.
  rank_zero_warn(

2023-05-24 03:21:32,272 - {pytorch_tabular.tabular_model:568} - INFO - Training the model completed
2023-05-24 03:21:32,273 - {pytorch_tabular.tabular_model:1207} - INFO - Loading the best model


In [90]:
pred_df = tabular_model.predict(test)

### Test accuracy
print((pred_df['y'] == pred_df['prediction']).mean())

### Test MSE
print(((pred_df['y'] - pred_df['prediction'])**2).mean())

Output()

0.6
0.4857142857142857
